# Comparison of cities - New York vs. Toronto

### Capstone Project by Enok Johannes Haahr Skjølstrup

## 1. Introduction / Business Problem

In this project, I have decided to compare the cities New York and Toronto. The comparison includes the size of the cities, number of borough, neighborhoods, and inhabitants, as well as a detailed description of venues. For example it is examined, how many restaurants, bars, museums, and sport fascilities that are found in each city, and how they are distributed in the cities. As New York is larger than Toronto, the number of different venues are scaled to the number of inhabitants and city area. The project can be of particular interest for tourists, considering which city they want to visit.

## 2. Data

In general, the Foursquare location data is applied to solve the problem. The data is stored in Pandas dataframes, and the Folium package is used to create plots of the two cities.
The data of New York is downloaded as a json file, while the data regarding the boroughs and neighborhoods of Toronto are taken from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M. 
The coordinates are afterwards loaded as a csv file.
The Foursquare is applied to search for venues in the two cities


### Load packages

In [1]:
%reset
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0



geopy-1.22.0         | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: 

### Load data

Toronto data

In [2]:
d = {'Postal Code': ['M3A', 'M4A','M5A','M6A','M7A','M9A','M1B','M3B','M4B','M5B','M6B','M9B','M1C','M3C','M4C','M5C','M6C',
                    'M9C','M1E','M4E','M5E','M6E','M1G','M4G','M5G','M6G','M1H','M2H','M3H','M4H','M5H','M6H','M1J','M2J','M3J',
                    'M4J','M5J','M6J','M1K','M2K','M3K','M4K','M5K','M6K','M1K','M2L','M3L','M4L','M5L','M6L','M9L','M1M',
                    'M2M','M3M','M4M','M5M','M6M','M9M','M1N','M2N','M3N','M4N','M5N','M6N','M9N','M1P','M2P','M4P','M5P',
                     'M6P','M9P','M1R','M2R','M4R','M5R','M6R','M7R','M9R','M1S','M4S','M5S','M6S','M1T','M4T','M5T','M1V',
                    'M4V','M5V','M8V','M9V','M1W','M4W','M5W','M8W','M9W','M1X','M4X','M5X','M8X','M4Y','M7Y','M8Y','M8Z'], 
'Borough': ['North York', 'North York','Downtown Toronto','North York','Downtown Toronto','Etobicoke','Scarborough',
        'North York','East York','Downtown Toronto','North York','Etobicoke','Scarborough','North York','East York',
        'Downtown Toronto','York','Etobicoke','Scarborough','East Toronto','Downtown Toronto','York','Scarborough','Eastside',
        'Downtown Toronto','Downtown Toronto','Scarborough','North York','North York','East York','Downtown Toronto',
        'West Toronto','Scarborough','North York','North York','East York','Downtown Toronto','West Toronto','Scarborough',
        'North York','North York','East Toronto','Downtown Toronto','West Toronto','Scarborough','North York','North York',
        'East Toronto','Downtown Toronto','North York','North York','Scarborough','North York','North York','East Toronto',
        'North York','York','North York','Scarborough','North York','North York','Central Toronto','Central Toronto','York',
        'York','Scarborough','North York','Central Toronto','Central Toronto','West Toronto','Etobicoke','Scarborough','North York',
        'Central Toronto','Central Toronto','West Toronto','Mississauga','Etobicake','Scarborough','Central Toronto','Downtown Toronto',
        'West Toronto','Scarborough','Central Toronto','Downtown Toronto','Scarborough','Central Toronto','Downtown Toronto','Etobicoke','Etobicoke','Scarborough','Downtown Toronto','Downtown Toronto','Etobicoke',
        'Etobicoke','Scarborough','Downtown Toronto','Downtown Toronto','Etobicoke','Downtown Toronto','East Toronto','Etobicoke','Etobicoke'] , 
'Neighborhood': ['Parkwoods', 'Victoria Village','Regent Park, Harbourfront','Lawrence Manor, Lawrence Heights',
        'Queens Park, Ontario Provincial Government','Islington Avenue','Malvern, Rouge','Don Mills','Parkview Hill, Woodbine Gardens',
        'Garden District, Ryerson','Glencairn','West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale',
        'Rouge Hill, Port Union, Highland Creek','Don Mills','Woodbine Heights','St. James Town',
        'Humewood-Cedarvale','Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood','Guildwood, Morningside, West Hill',
        'The Beaches','Berczy Park','Caledonia-Fairbanks','Woburn','Leaside','Central Bay Street','Christie','Cedarbrae',
        'Hillcrest Village','Bathurst Manor, Wilson Heights, Downsview North','Thorncliffe Park','Richmond, Adelaide, King',
        'Dufferin, Dovercourt Village','Scarborough Village','Fairview, Henry Farm, Oriole','Northwood Park, York University',
        'East Toronto','Harbourfront East, Union Station, Toronto Islands','Little Portugal, Trinity','Kennedy Park, Ionview, East Birchmount Park',
         'Bayview Village','Downsview','The Danforth West, Riverdale','Toronto Dominion Centre, Design Exchange','Brockton, Parkdale Village, Exhibition Place',
        'Golden Mile, Clairlea, Oakridge','York Mills, Silver Hills','Downsview','India Bazaar, The Beaches West','Commerce Court, Victoria Hotel',
        'North Park, Maple Leaf Park, Upwood Park','Humber Summit','Cliffside, Cliffcrest, Scarborough Village West','Willowdale, Newtonbrook',
        'Downsview','Studio District','Bedford Park, Lawrence Manor East','Del Ray, Mount Dennis, Keelsdale and Silverthorn',
        'Humberlea, Emery','Birch Cliff, Cliffside West','Willowdale','Downsview','Lawrence Park','Roselawn','Runnymede, The Junction North',
        'Weston','Dorset Park, Wexford Heights, Scarborough Town Centre','York Mills West','Davisville North','Forest Hill North & West',
        'High Park, The Junction South','Westmount','Wexford, Maryvale','Willowdale','North Toronto West','The Annex, North Midtown, Yorkville',
        'Parkdale, Roncesvalles','Canada Post Gateway Processing Centre','Kingsview Village, St Phillips, Martin Grove Gardens, Richview Gardens','Agincourt',
        'Davisville','University of Toronto, Harbord','Runnymede, Swansea','Clarks Corners, Tam OShanter, Sullivan','Moore Park, Summerhill East',
        'Kensington Market, Chinatown, Grange Park','Milliken, Agincourt North, Steeles East, LAmoreaux East',
        'Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park','CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport',
        'New Toronto, Mimico South, Humber Bay Shores','South Steeles, Silverstone, Humbergate, Jamestown, Mount Olive, Beaumond Heights, Thistletown, Albion Gardens',
        'Steeles West, L Amoreaux West','Rosedale','Stn A PO Boxes','Alderwood, Long Branch','Northwest','Upper Rouge',
        'St James Town, Cabbagetown','First Canadian Place, Underground city','The Kingsway, Montgomery Road, Old Mill North',
        'Church and Wellesley','Business reply mail Processing Centre','Old Mill South, Kings Mill Park, Sunnylea, Humber Bay, Mimico NE, The Queensway East, Royal York South East, Kingsway Park South East',
        'Mimico NW, The Queensway West, South of Bloor, Kingsway Park South West, Royal York South West'
]}

toronto_data = pd.DataFrame(data=d)

The coordinates are given in a csv file that is loaded

In [3]:
coord=pd.read_csv('Geospatial_Coordinates_Toronto.csv') #Read the csv file into a pandas dataframe

Lat=coord['Latitude'];
Long=coord['Longitude'];

In [4]:
toronto_data=toronto_data.sort_values(by=['Postal Code']); #The values in the df dataframe are also sorted after postal code
toronto_data['Latitude']=Lat; #Latitude and longitude are added to the dataframe
toronto_data['Longitude']=Long;

toronto_data=toronto_data.drop(['Postal Code'],axis=1) #The postal code is dropped as it is not comparable to the New York data anyway

In [5]:
toronto_data.head()

,Borough,Neighborhood,Latitude,Longitude
6,Scarborough,"Malvern, Rouge",43.727929,-79.262029
12,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.794200,-79.262029
18,Scarborough,"Guildwood, Morningside, West Hill",43.778517,-79.346556
22,Scarborough,Woburn,43.770120,-79.408493
26,Scarborough,Cedarbrae,43.745906,-79.352188


New York data

In [6]:
!wget -q -O 'newyork.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [7]:
with open('newyork.json') as json_data:
    newyork = json.load(json_data)

In [8]:
neighborhoods_data = newyork['features'] #The neighborhood data is found in the feaures of the data

# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
newyork_data = pd.DataFrame(columns=column_names)
newyork_data

,Borough,Neighborhood,Latitude,Longitude


The data is added in a loop

In [9]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    newyork_data = newyork_data.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [10]:
newyork_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


### Data for airports are added

New York

In [11]:
JFK={'Borough': 'Airport','Neighborhood': 'JFK', 'Latitude': 40.6441666667,'Longitude': -73.7822222222} #Data from https://www.travelmath.com/airport/JFK
Newark={'Borough': 'Airport','Neighborhood': 'Newark', 'Latitude': 40.7026321895,'Longitude': -74.1877659156} #Data from https://latitude.to/articles-by-country/us/united-states/27243/newark-liberty-international-airport-njt-station
LaGuardia={'Borough': 'Airport','Neighborhood': 'LaGuardia', 'Latitude': 40.7769271,'Longitude': -73.87396590000003} #Data from https://www.distancesto.com/coordinates/us/laguardia-airport-lga-latitude-longitude/history/3898.html

newyork_data=newyork_data.append([JFK,Newark,LaGuardia],ignore_index=True)

In [12]:
print('Shape of New York dataframe:',newyork_data.shape)
print('New York has {} boroughs, {} neighborhoods, and {} airports.'.format(
        len(newyork_data['Borough'].unique())-1,
        newyork_data.shape[0]-sum(newyork_data['Borough']=='Airport'), sum(newyork_data['Borough']=='Airport')
    )
)

Shape of New York dataframe: (309, 4)
New York has 5 boroughs, 306 neighborhoods, and 3 airports.


Toronto

In [13]:
Pearson={'Borough': 'Airport','Neighborhood': 'Pearson', 'Latitude': 43.67771760000001,'Longitude': -79.62481969999999} #Data from https://www.distancesto.com/coordinates/ca/toronto-pearson-international-latitude-longitude/history/3535.html

toronto_data=toronto_data.append(Pearson,ignore_index=True)

In [14]:
print('Shape of Toronto dataframe:',toronto_data.shape)
print('Toronto has {} boroughs, {} neighborhoods, and {} airport.'.format(
        len(toronto_data['Borough'].unique())-1,
        toronto_data.shape[0]-sum(toronto_data['Borough']=='Airport'), sum(toronto_data['Borough']=='Airport')
    )
)

Shape of Toronto dataframe: (104, 4)
Toronto has 12 boroughs, 103 neighborhoods, and 1 airport.


### Population data is introduced

In [15]:
toronto_area=630.2 #area in km^2. from https://en.wikipedia.org/wiki/Toronto
toronto_population= 2731571 #from https://en.wikipedia.org/wiki/Toronto
    
newyork_area=784 #area in km^2. from https://en.wikipedia.org/wiki/New_York_City
newyork_population=8336817 #from https://en.wikipedia.org/wiki/New_York_City

### Plot data

## Plot map of Toronto (airport is red)

In [16]:
neigh_toronto = 'Woburn' #Set a neighborhood

idx=toronto_data.index[toronto_data['Neighborhood'] == neigh_toronto];
latitude=toronto_data.at[idx[0],'Latitude'] #The latitude for the postal code
longitude=toronto_data.at[idx[0],'Longitude'] #The longitude for the postal code

print('The geograpical coordinate of', neigh_toronto, 'are',latitude,longitude)



The geograpical coordinate of Woburn are 43.7701199 -79.40849279999999


In [17]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    if borough=='Airport':
        label = '{}, {}'.format(neighborhood, borough)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='#ff0000',
            fill_opacity=0.7,
            parse_html=False).add_to(map_toronto)  
    
map_toronto

## Plot map of New York (airports are red)

In [18]:
neigh_newyork = 'Brooklyn Heights' #Set a neighborhood

idx=newyork_data.index[newyork_data['Neighborhood'] == neigh_newyork];
latitude=newyork_data.at[idx[0],'Latitude'] #The latitude for the postal code
longitude=newyork_data.at[idx[0],'Longitude'] #The longitude for the postal code

print('The geograpical coordinate of', neigh_newyork, 'are',latitude,longitude)



The geograpical coordinate of Brooklyn Heights are 40.695863722724084 -73.99378225496424


The airports are plotted in red

In [19]:
# create map of Toronto using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)
color='blue',
# add markers to map
for lat, lng, borough, neighborhood in zip(newyork_data['Latitude'], newyork_data['Longitude'], newyork_data['Borough'], newyork_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    if borough=='Airport':
        label = '{}, {}'.format(neighborhood, borough)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='#ff0000',
            fill_opacity=0.7,
            parse_html=False).add_to(map_newyork)  
    
    
map_newyork

#### Foursquare credentials

The Foursquare credentials are used in the next part of the project to obtain all the venues near a given location

In [20]:
CLIENT_ID = '0QPHQ3C450TRXRDK4G4UIQYVGJYRYZSWWX0KHIEMSKN2N0LN' # your Foursquare ID
CLIENT_SECRET = 'KPS0NWUBYEVT2UFTBFU5PYHSVK2JVBR2FDDB32MIQGZ14T2N' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0QPHQ3C450TRXRDK4G4UIQYVGJYRYZSWWX0KHIEMSKN2N0LN
CLIENT_SECRET:KPS0NWUBYEVT2UFTBFU5PYHSVK2JVBR2FDDB32MIQGZ14T2N


In [22]:
#jupyter trust mynotebook.ipynb
